In [ ]:
!pip install langchain
!pip install langchain_community
!pip install openai
!pip install tiktoken

In [2]:
%%writefile qwen.env
## Qwen base_url
OPENAI_BASE_URL='https://dashscope.aliyuncs.com/compatible-mode/v1'
DASHSCOPE_API_KEY='sk-d4c3f773af5d442c97eb7d9e40479c55'
OPENAI_API_KEY=${DASHSCOPE_API_KEY}

Writing qwen.env


In [3]:
## 使用qwen key
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv('qwen.env',override=True)

import os
print(os.environ['OPENAI_BASE_URL'],os.environ['OPENAI_API_KEY'])

https://dashscope.aliyuncs.com/compatible-mode/v1 sk-d4c3f773af5d442c97eb7d9e40479c55


## langchains.memory
LangChain 提供了多种储存类型。其中，
- 缓冲区储存允许保留最近的**聊天消息**
- 摘要储存则提供了对**整个对话的摘要**。
- 实体储存则允许在多轮对话中保留有关**特定实体的信息**。
这些记忆组件都是模块化的，可与其他组件组合使用，从而增强机器人的对话管理能力。

### 四种常用的存储模块
**使用方法：**建立一个存储模块，然后传参到ConversationChain中即可。

#### 1.对话缓存储存 (buffer.ConversationBufferMemory）
当我们在使用大型语言模型进行聊天对话时，大型语言模型本身实际上是无状态的。语言模型本身并不记得到目前为止的历史对话。每次调用API结点都是独立的。**储存(Memory)可以储存到目前为止的所有术语或对话，并将其输入或附加上下文到LLM中用于生成输出**。如此看起来就好像它在进行下一轮对话的时候，记得之前说过什么。

#### 2.对话缓存窗口储存 (buffer_window.ConversationBufferWindowMemory）
随着对话变得越来越长，所需的内存量也变得非常长。将大量的tokens发送到LLM的成本，也会变得更加昂贵，这也就是为什么API的调用费用，通常是基于它需要处理的tokens数量而收费的。针对以上问题，LangChain也提供了几种方便的储存方式来保存历史对话。其中，对话缓存窗口储存只保留一个窗口大小的对话。**它只使用最近的n次交互。这可以用于保持最近交互的滑动窗口，以便缓冲区不会过大**。

#### 3.对话令牌缓存储存 (token_buffer.ConversationTokenBufferMemory）
使用对话字符缓存记忆，**内存将限制保存的token数量**。如果字符数量超出指定数目，它会切掉这个对话的早期部分 以保留与最近的交流相对应的字符数量，但不超过字符限制。

#### 4.对话摘要缓存储存 (summary.ConversationSummaryBufferMemory）
对话摘要缓存储存，**使用 LLM 对到目前为止历史对话自动总结摘要**，并将其保存下来。

#### ConversationBufferMemory

In [35]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory.buffer import ConversationBufferMemory

model_name='qwen2-7b-instruct'
llm = ChatOpenAI(model=model_name,temperature=0.0)  

memory = ConversationBufferMemory()

# 新建一个 ConversationChain Class 实例
# verbose参数设置为True时，程序会输出更详细的信息，以提供更多的调试或运行时信息。
# 相反，当将verbose参数设置为False时，程序会以更简洁的方式运行，只输出关键的信息。
conversation = ConversationChain(llm=llm, memory = memory, verbose=True)
conversation

ConversationChain(verbose=True, llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f142d00be50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f142d01e560>, model_name='qwen2-7b-instruct', temperature=0.0, openai_api_key='sk-d4c3f773af5d442c97eb7d9e40479c55', openai_proxy=''))

In [36]:
## 进行多轮对话
conversation.verbose=False
output=conversation.predict(input="你好, 我叫皮皮鲁")
#print(output)
output=conversation.predict(input="1+1等于多少？")
#print(output)
conversation.verbose=True
output=conversation.predict(input="我叫什么名字？")
print(output)

print('memory:')
print(conversation.memory.buffer)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好, 我叫皮皮鲁
AI: 你好，皮皮鲁！很高兴见到你。作为一个AI，我能够回答各种问题，提供信息和进行对话。有什么我可以帮助你的吗？
Human: 1+1等于多少？
AI: 1+1等于2。
Human: 我叫什么名字？
AI:

> Finished chain.
你刚才提到你叫皮皮鲁。所以，你的名字是皮皮鲁。
memory:
Human: 你好, 我叫皮皮鲁
AI: 你好，皮皮鲁！很高兴见到你。作为一个AI，我能够回答各种问题，提供信息和进行对话。有什么我可以帮助你的吗？
Human: 1+1等于多少？
AI: 1+1等于2。
Human: 我叫什么名字？
AI: 你刚才提到你叫皮皮鲁。所以，你的名字是皮皮鲁。


In [10]:
print(memory.load_memory_variables({}))

{'history': 'Human: 你好, 我叫皮皮鲁\nAI: 你好，皮皮鲁！很高兴见到你。作为一个AI，我能够回答各种问题，提供信息和进行对话。有什么我可以帮助你的吗？\nHuman: 1+1等于多少？\nAI: 1+1等于2。这是最基本的数学加法。\nHuman: 我叫什么名字？\nAI: 你刚才说你叫皮皮鲁。所以，你的名字是皮皮鲁。'}


In [37]:
## 也可手动将对话添加到memory中
memory = ConversationBufferMemory()
memory.save_context({"input": "你好，我叫皮皮鲁"}, {"output": "你好啊，我叫鲁西西"})
memory.save_context({"input": "很高兴和你成为朋友！"}, {"output": "是的，让我们一起去冒险吧！"})
memory.load_memory_variables({})


{'history': 'Human: 你好，我叫皮皮鲁\nAI: 你好啊，我叫鲁西西\nHuman: 很高兴和你成为朋友！\nAI: 是的，让我们一起去冒险吧！'}

#### ConversationBufferWindowMemory

In [44]:
from langchain.memory.buffer_window import ConversationBufferWindowMemory

# k=1表明只保留一个对话记忆
memory = ConversationBufferWindowMemory(k=1)  
memory.save_context({"input": "你好，我叫皮皮鲁"}, {"output": "你好啊，我叫鲁西西"})
memory.save_context({"input": "很高兴和你成为朋友！"}, {"output": "是的，让我们一起去冒险吧！"})
memory.load_memory_variables({})


{'history': 'Human: 很高兴和你成为朋友！\nAI: 是的，让我们一起去冒险吧！'}

In [39]:
llm = ChatOpenAI(model=model_name,temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(llm=llm, memory=memory, verbose=False  )

print("第一轮对话：")
print(conversation.predict(input="你好, 我叫皮皮鲁"))

print("第二轮对话：")
print(conversation.predict(input="1+1等于多少？"))

print("第三轮对话：")
print(conversation.predict(input="我叫什么名字？"))


第一轮对话：
你好，皮皮鲁！很高兴见到你。作为一个AI，我能够帮助你解答问题、提供信息或者进行有趣的对话。请问有什么我可以帮助你的吗？
第二轮对话：
1+1等于2。这是最基本的数学加法。
第三轮对话：
作为一个AI，我并没有直接访问你个人信息的能力，因此无法告诉你你的名字。如果你愿意告诉我，我很乐意听到。不过，记得个人信息的安全和隐私是非常重要的哦！


#### ConversationTokenBufferMemory

In [53]:
from langchain.llms import OpenAI
from langchain.memory.token_buffer import ConversationTokenBufferMemory
llm=ChatOpenAI(
    model='gpt-3.5-turbo',
    temperature=0.0,
    base_url='https://threefive.gpt7.link/v1',
    openai_api_key='sk-7INmRu6iHEope6cJCa462b608aFb4eE281Df64F4E32e95F9'
)  
## 只会保留最后30个tokens
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "朝辞白帝彩云间，"}, {"output": "千里江陵一日还。"})
memory.save_context({"input": "两岸猿声啼不住，"}, {"output": "轻舟已过万重山。"})
memory.load_memory_variables({})


{'history': 'AI: 轻舟已过万重山。'}

#### ConversationSummaryBufferMemory

In [61]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory.summary_buffer import ConversationSummaryBufferMemory

# 创建一个长字符串
schedule = "在八点你和你的产品团队有一个会议。 \
你需要做一个PPT。 \
上午9点到12点你需要忙于LangChain。\
Langchain是一个有用的工具，因此你的项目进展的非常快。\
中午，在意大利餐厅与一位开车来的顾客共进午餐 \
走了一个多小时的路程与你见面，只为了解最新的 AI。 \
确保你带了笔记本电脑可以展示最新的 LLM 样例."

llm=ChatOpenAI(
    model='gpt-3.5-turbo',
    temperature=0.0,
    base_url='https://threefive.gpt7.link/v1',
    openai_api_key='sk-7INmRu6iHEope6cJCa462b608aFb4eE281Df64F4E32e95F9'
)  
## 设置summary的最大长度
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "你好，我叫皮皮鲁"}, {"output": "你好啊，我叫鲁西西"})
memory.save_context({"input": "很高兴和你成为朋友！"}, {"output": "是的，让我们一起去冒险吧！"})
memory.save_context({"input": "今天的日程安排是什么？"}, {"output": f"{schedule}"})

print(memory.load_memory_variables({})['history'])


System: The human introduces themselves as 皮皮鲁, and the AI introduces itself as 鲁西西. They express happiness at becoming friends and discuss their agenda for the day. The AI informs the human about a meeting with their product team at 8 AM, during which they need to prepare a PPT. It mentions that from 9 AM to 12 PM, the human will be busy with LangChain, a useful tool that significantly aids their project’s progress. At noon, the human has a lunch meeting with a customer at an Italian restaurant, where they will discuss the latest in AI, reminding them to bring a laptop to showcase the newest LLM examples.


In [63]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)
conversation.predict(input="展示什么样的样例最好呢？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human introduces themselves as 皮皮鲁, and the AI introduces itself as 鲁西西. They express happiness at becoming friends and discuss their agenda for the day. The AI informs the human about a meeting with their product team at 8 AM, during which they need to prepare a PPT. It mentions that from 9 AM to 12 PM, the human will be busy with LangChain, a useful tool that significantly aids their project’s progress. At noon, the human has a lunch meeting with a customer at an Italian restaurant, where they will discuss the latest in AI, reminding them to bring a laptop to showcase the newest LLM examples.
Human: 展示什么样的样例最好呢？
AI:

> Finished chain.


'展示最新的大型语言模型（LLM）示例时，可以考虑以下几种类型的样例：\n\n1. **对话生成**：展示模型如何进行自然对话，比如模拟客服对话或伙伴聊天，这能让客户看到其在自然语言处理中的应用。\n\n2. **文本摘要**：使用模型对长篇文章进行自动摘要，快速提炼关键信息，展示其在信息处理方面的能力。\n\n3. **内容生成**：展示模型如何生成创意写作、新闻报道或技术文档，突出其创造性和适应性。\n\n4. **问答系统**：演示模型如何回答特定领域的问题，例如医疗、法律或科技相关的问题，体现其专业性和准确性。\n\n5. **翻译实例**：展示模型提供的翻译服务，突出其多语言支持能力，这在国际化项目中尤为重要。\n\n确保准备好相关的实际案例或数据，以便在展示时具体说明模型的准确性和实用性！如果你还有其他想法或者需要进一步的建议，随时告诉我哦！'

In [64]:
## 摘要记录会自动更新
print(memory.load_memory_variables({}))  

{'history': "System: The human introduces themselves as 皮皮鲁, and the AI introduces itself as 鲁西西. They express happiness at becoming friends and discuss their agenda for the day, including a product team meeting at 8 AM where they need to prepare a PPT. The human will be busy with LangChain from 9 AM to 12 PM, and they have a lunch meeting with a customer at an Italian restaurant at noon to discuss AI, where the human needs to bring their laptop to showcase the latest LLM examples. The human inquires about the best types of examples to present, and the AI suggests several, including dialogue generation, text summarization, content creation, question-answering systems, and translation examples, emphasizing the importance of having actual cases or data to illustrate the model's precision and practicality during the presentation."}
